In [1]:
# setup Jupyter version of Dash
from jupyter_dash import JupyterDash

# configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# configure OS routines
import os

# configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### FIX ME ####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from script import AnimalShelter 

#############################
# data manipulation / model
#############################
# FIX ME - update with your username and password and CRUD Python module name
username = "aacuser"
password = "password"

# connect to database via CRUD module
db = AnimalShelter(username,password)

# class read method must support return of list object and accept projection json input
df = pd.DataFrame.from_records(db.read({}))

# remove '_id' column to avoid data table crash
df.drop(columns=['_id'], inplace=True)

#############################
# dashboard layout / view
#############################
app = JupyterDash(__name__)

# FIX ME - add in Grazioso Salvare's logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())
                                 
# FIX ME - place the html image tag in the line below into the app.layout code according to your design
# FIX ME - also remember to include a unique identifier such as your name or date
app.layout = html.Div([
    # display organization logo
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width': '200px'}),
    html.Center(html.B(html.H1('CS-340 Dashboard - Trinity Anderson'))),
    html.Hr(),
    
    # FIX ME - add in code for the interactive filtering options
    html.Div(
        # radio buttons allows users to filter by rescue category
        dcc.RadioItems(
            id = 'filter-type',
            options =[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'wilderness'},
                {'label': 'Disaster Rescue/Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset',
            labelStyle={'display': 'block'}
        )
    ),
    
    html.Hr(),
    
    # display interactive data table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    
    html.Br(),
    html.Hr(),
    
    # layout for graph and map charts side-by-side
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])
                                 
###############################################
# interaction between components / controller
###############################################
# filter MongoDB results based on user-selected rescue type
@app.callback(Output('datatable-id', 'data'),
             [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    # use hardcoded breed/age/sex filters from dashboard specifications
    if filter_type == 'water':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'wilderness':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'disaster':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
    else:
        query = {}
                                 
    results = db.read(query)
    filtered_df = pd.DataFrame.from_records(results)
    if '_id' in filtered_df.columns:
        filtered_df.drop(columns=['_id'], inplace=True)
    return filtered_df.to_dict('records')

# chart of breed distribution using plotly pie chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        return [html.P("No data available for chart.")]
                                 
    dff = pd.DataFrame.from_dict(viewData)
    fig = px.pie(dff, names='breed', title='Breed Distribution')
    return [dcc.Graph(figure=fig)]
                                 
# highlight column selection
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if':{'column_id': i},
        'background_color': '#D3F3ff'
    } for i in selected_columns]

# geolocation map based on data table row selection
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
    Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or index is None or len(viewData) == 0:
        return [html.P("No data to display on map.")]
                                 
    dff = pd.DataFrame.from_dict(viewData)
    if len(dff.columns) < 14:
        return [html.P("Incomplete data for map display.")]
                                 
    row = index[0] if index else 0
                                 
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row,13], dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]

# run server
app.run_server(debug=True)

Dash app running on http://127.0.0.1:31436/
